In [190]:
from datetime import datetime, timedelta
from getpass import getpass
import MetaTrader5 as mt5
import pandas as pd
import numpy as np
import statistics

In [298]:
def f_leer_archivo():
    Accounts = pd.read_excel('files/Accounts.xlsx')
    Names = list(Accounts['Name'])
    print('Los nombres son:',Names)
    Name = input('Ingrese nombre del integrante a analizar: ')
    
    if Name in Names:
        user = Accounts[Accounts['Name']==Name]
        try:
            userid = int(user['Account_ID'].values[0])
            userpass = str(user['Password'].values[0])
        except:
            try:
                userid = int(input('Ingrese su usuario en mt5: '))
                userpass = str(getpass('Ingrese su contraseña: '))
            except:
                return print('La acción no fue válida, verifique sus datos')
        #Initialize MT5
        a = mt5.initialize(login=userid, server='MetaQuotes-Demo',password=userpass)
        if a:
            print('Se inició con éxito la cuenta')
            pos = mt5.history_deals_get(datetime(2020,1,1),datetime.now())
            key = pos[0]._asdict().keys()
            orders = pd.DataFrame(list(pos),columns=key)
            
            Dates = pd.DataFrame({'Position':orders['position_id'].unique()})
            opentime = []
            closetime = []
            price_o = []
            price_c = []
            for i in orders['position_id'].unique():
                dates = np.array(orders['time'][orders['position_id']==i])
                prices = np.array(orders['price'][orders['position_id']==i])
                if len(dates)==2:
                    opentime.append(dates[0])
                    closetime.append(dates[1])
                    price_o.append(prices[0])
                    price_c.append(prices[1])
                else:
                    opentime.append(dates[0])
                    closetime.append(0)
                    price_o.append(prices[0])
                    price_c.append(prices[-1])

            Dates['Time'] = opentime
            Dates['Symbol'] = [np.array(orders['symbol'][orders['position_id']==i])[0] for i in orders['position_id'].unique()]
            type_op = [np.array(orders['type'][orders['position_id']==i])[0] for i in orders['position_id'].unique()]
            Dates['Type'] = ['buy' if i == 0 else 'sell' for i in type_op]
            Dates['Volume'] = [np.array(orders['volume'][orders['position_id']==i])[0] for i in orders['position_id'].unique()]
            Dates['Price'] = price_o
            Dates['Time.1'] = closetime
            Dates['Price.1'] = price_c
            Dates['Commission'] = [np.array(orders['commission'][orders['position_id']==i])[-1] for i in orders['position_id'].unique()]
            Dates['Swap'] = [np.array(orders['swap'][orders['position_id']==i])[-1] for i in orders['position_id'].unique()]
            Dates['Profit'] = [np.array(orders['profit'][orders['position_id']==i])[-1] for i in orders['position_id'].unique()]
            Dates = Dates[Dates['Time.1']!=0]
            return Dates.sort_values(by='Time.1',ascending=True).reset_index(drop=True)
            
        else:
            print('No se pudo acceder a la cuenta')
    else:
        print('El nombre no fue válido, intente de nuevo')
        
        
#Time open, $position, Symbol, Type, Volume, Price, S/L, T/P Time close, Price clsoe, Comission, swap, profit

In [299]:
data = f_leer_archivo()
data

Los nombres son: ['Fernando', 'Sofia', 'Otro']
Ingrese nombre del integrante a analizar: Otro
Ingrese su usuario en mt5: kxkxkx
La acción no fue válida, verifique sus datos


In [286]:
def f_pip_size(param_ins):
    try:
        mt5.initialize(login=41671538, server='MetaQuotes-Demo',password='pcdt7hng')
        pip_size = int(0.1/mt5.symbol_info(param_ins)._asdict().get('trade_tick_size'))
        return pip_size
    except:
        return print('Ticker no válido')

In [287]:
f_pip_size('USDJPY')

100

In [288]:
def f_columnas_tiempos(param_data):
    param_data['open_time'] = [datetime.fromtimestamp(i) for i in param_data['Time']]
    param_data['close_time'] = [datetime.fromtimestamp(i) for i in param_data['Time.1']]
    param_data['time'] = (param_data['close_time']-param_data['open_time']).apply(timedelta.total_seconds,1)
    return param_data

In [289]:
data2 = f_columnas_tiempos(data)
data2

,Position,Time,Symbol,Type,Volume,Price,Time.1,Price.1,Commission,Swap,Profit,open_time,close_time,time
0,873210071,1613722239,EURUSD,buy,0.10,1.20965,1613726270,1.21066,0.0,0.00,10.10,2021-02-19 02:10:39,2021-02-19 03:17:50,4031.0
1,874804645,1613965942,EURCHF,buy,1.08,1.08747,1613987760,1.08945,0.0,0.00,237.67,2021-02-21 21:52:22,2021-02-22 03:56:00,21818.0
2,876896332,1614070306,EURUSD,buy,10.00,1.21672,1614076580,1.21565,0.0,0.00,-1070.00,2021-02-23 02:51:46,2021-02-23 04:36:20,6274.0
3,878179027,1614114199,USDCHF,buy,10.00,0.90353,1614114879,0.90368,0.0,0.00,165.99,2021-02-23 15:03:19,2021-02-23 15:14:39,680.0
4,878190806,1614114869,GBPUSD,buy,10.00,1.41096,1614128997,1.41138,0.0,-2.00,420.00,2021-02-23 15:14:29,2021-02-23 19:09:57,14128.0
5,880463983,1614206738,AUDCAD,buy,1.00,0.99669,1614217567,0.99767,0.0,15.34,78.33,2021-02-24 16:45:38,2021-02-24 19:46:07,10829.0
6,880524766,1614210221,GBPJPY,buy,2.00,149.69400,1614250900,150.40000,0.0,1.70,1330.32,2021-02-24 17:43:41,2021-02-25 05:01:40,40679.0
7,882721329,1614309825,EURCHF,sell,1.10,1.10073,1614332798,1.09770,0.0,0.00,368.45,2021-02-25 21:23:45,2021-02-26 03:46:38,22973.0
8,884420278,1614370053,CADJPY,buy,2.79,83.90000,1614557756,83.58800,0.0,-2.88,-816.75,2021-02-26 14:07:33,2021-02-28 18:15:56,187703.0
9,884819580,1614574064,EURCHF,buy,7.30,1.09788,1614597614,1.09681,0.0,0.00,-857.53,2021-02-28 22:47:44,2021-03-01 05:20:14,23550.0


In [290]:
def f_columnas_pips(param_data):
    
    param_data['pips'] = [(param_data['Price.1'].iloc[i]-param_data['Price'].iloc[i])*f_pip_size(param_data['Symbol'].iloc[i]) 
                          if param_data['Type'].iloc[i]=='buy' 
                          else (param_data['Price'].iloc[i]-param_data['Price.1'].iloc[i])*f_pip_size(param_data['Symbol'].iloc[i]) 
                          for i in range(len(param_data))]
    param_data['pips_acm'] = param_data['pips'].cumsum()
    param_data['profit_acm'] = param_data['Profit'].cumsum()
    return param_data

In [291]:
data3 = f_columnas_pips(data2)
data3

,Position,Time,Symbol,Type,Volume,Price,Time.1,Price.1,Commission,Swap,Profit,open_time,close_time,time,pips,pips_acm,profit_acm
0,873210071,1613722239,EURUSD,buy,0.10,1.20965,1613726270,1.21066,0.0,0.00,10.10,2021-02-19 02:10:39,2021-02-19 03:17:50,4031.0,10.1,10.1,10.10
1,874804645,1613965942,EURCHF,buy,1.08,1.08747,1613987760,1.08945,0.0,0.00,237.67,2021-02-21 21:52:22,2021-02-22 03:56:00,21818.0,19.8,29.9,247.77
2,876896332,1614070306,EURUSD,buy,10.00,1.21672,1614076580,1.21565,0.0,0.00,-1070.00,2021-02-23 02:51:46,2021-02-23 04:36:20,6274.0,-10.7,19.2,-822.23
3,878179027,1614114199,USDCHF,buy,10.00,0.90353,1614114879,0.90368,0.0,0.00,165.99,2021-02-23 15:03:19,2021-02-23 15:14:39,680.0,1.5,20.7,-656.24
4,878190806,1614114869,GBPUSD,buy,10.00,1.41096,1614128997,1.41138,0.0,-2.00,420.00,2021-02-23 15:14:29,2021-02-23 19:09:57,14128.0,4.2,24.9,-236.24
5,880463983,1614206738,AUDCAD,buy,1.00,0.99669,1614217567,0.99767,0.0,15.34,78.33,2021-02-24 16:45:38,2021-02-24 19:46:07,10829.0,9.8,34.7,-157.91
6,880524766,1614210221,GBPJPY,buy,2.00,149.69400,1614250900,150.40000,0.0,1.70,1330.32,2021-02-24 17:43:41,2021-02-25 05:01:40,40679.0,70.6,105.3,1172.41
7,882721329,1614309825,EURCHF,sell,1.10,1.10073,1614332798,1.09770,0.0,0.00,368.45,2021-02-25 21:23:45,2021-02-26 03:46:38,22973.0,30.3,135.6,1540.86
8,884420278,1614370053,CADJPY,buy,2.79,83.90000,1614557756,83.58800,0.0,-2.88,-816.75,2021-02-26 14:07:33,2021-02-28 18:15:56,187703.0,-31.2,104.4,724.11
9,884819580,1614574064,EURCHF,buy,7.30,1.09788,1614597614,1.09681,0.0,0.00,-857.53,2021-02-28 22:47:44,2021-03-01 05:20:14,23550.0,-10.7,93.7,-133.42


In [292]:
def f_estadisticas_ba(param_data):
    Ops_totales = len(param_data)
    Ganadoras = len(param_data[param_data['Profit']>=0])
    Ganadoras_c = len(param_data[(param_data['Profit']>=0) & (param_data['Type']=='buy')])
    Ganadoras_v = len(param_data[(param_data['Profit']>=0) & (param_data['Type']!='buy')])
    Perdedoras = len(param_data[param_data['Profit']<0])
    Perdedoras_c = len(param_data[(param_data['Profit']<0) & (param_data['Type']=='buy')])
    Perdedoras_v = len(param_data[(param_data['Profit']<0) & (param_data['Type']!='buy')])
    Mediana_profit = statistics.median(param_data.sort_values(by='Profit')['Profit'])
    Mediana_pips = statistics.median(param_data.sort_values(by='pips')['pips'])
    r_efectividad = Ganadoras/Ops_totales
    r_proporcion = Ganadoras/Perdedoras
    r_efectividad_c = Ganadoras_c/Ops_totales
    r_efectividad_v = Ganadoras_v/Ops_totales
    
    valor = [Ops_totales,Ganadoras,Ganadoras_c,Ganadoras_v,Perdedoras,Perdedoras_c,Perdedoras_v,Mediana_profit,
             Mediana_pips,r_efectividad,r_proporcion,r_efectividad_c,r_efectividad_v]
    df_1_tabla = pd.DataFrame({'medida':['Ops totales','Ganadoras','Ganadoras_c','Ganadoras_v','Perdedoras',
                                         'Perdedoras_c','Perdedoras_v','Mediana(Profit)','Mediana(Pips)',
                                         'r_efectividad','r_proporcion','r_efectividad_c','r_efectividad_v'],
                               'valor':np.round(valor,2),
                               'descripcion':['Operaciones totales','Operaciones ganadoras',
                                              'Operaciones ganadoras de compra','Operaciones ganadoras de venta',
                                              'Operaciones perdedoras','Operaciones perdedoras de compra',
                                              'Operaciones perdedoras de venta','Mediana de profit de operaciones',
                                              'Mediana de pips de operaciones','Ganadoras Totales/Operaciones Totales',
                                              'Ganadoras Totales/Perdedoras Totales',
                                              'Ganadoras Compras/Operaciones Totales',
                                              'Ganadoras Ventas/Operaciones Totales']})
    symbols = param_data['Symbol'].unique()
    df_2_ranking = pd.DataFrame({'symbol':param_data['Symbol'].unique(),
                                 'rank (%)':100*np.round([len(param_data[(param_data['Profit']>0) & 
                                                        (param_data['Symbol']==symbols[i])])/
                                         len(param_data[param_data['Symbol']==symbols[i]]) 
                                         for i in range(len(symbols))],2)
                                })
   
    df_2_ranking = df_2_ranking.sort_values(by='rank (%)',ascending=False).reset_index(drop=True)
    
    return {'df_1_tabla':df_1_tabla,'df_2_ranking':df_2_ranking}

In [293]:
data4 = f_estadisticas_ba(data3)

In [294]:
data4.get('df_1_tabla')

,medida,valor,descripcion
0,Ops totales,15.00,Operaciones totales
1,Ganadoras,10.00,Operaciones ganadoras
2,Ganadoras_c,6.00,Operaciones ganadoras de compra
3,Ganadoras_v,4.00,Operaciones ganadoras de venta
4,Perdedoras,5.00,Operaciones perdedoras
5,Perdedoras_c,4.00,Operaciones perdedoras de compra
6,Perdedoras_v,1.00,Operaciones perdedoras de venta
7,Mediana(Profit),165.99,Mediana de profit de operaciones
8,Mediana(Pips),7.60,Mediana de pips de operaciones
9,r_efectividad,0.67,Ganadoras Totales/Operaciones Totales


In [295]:
data4.get('df_2_ranking')

,symbol,rank (%)
0,USDCHF,100.0
1,AUDCAD,100.0
2,GBPJPY,100.0
3,GBPCHF,100.0
4,EURUSD,67.0
5,EURCHF,50.0
6,GBPUSD,50.0
7,CADJPY,0.0


In [300]:
a = mt5.initialize(login=41740782, server='MetaQuotes-Demo',password='0xefeffr')

In [305]:
key = mt5.history_deals_get(datetime(2020,1,1),datetime.now())[0]._asdict().keys()

In [306]:
pos = mt5.history_deals_get(datetime(2020,1,1),datetime.now())
key = pos[0]._asdict().keys()
orders = pd.DataFrame(list(pos),columns=key)